# Model Generation

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import configparser
import pandas as pd

import torch
import pytorch_lightning as pl

from pynas.core.population import Population
from datasets.RawClassifier.loader import RawClassifierDataModule

# Define dataset module
root_dir = '/Data_large/marine/PythonProjects/OtherProjects/lpl-PyNas/data/RawClassifier'
dm = RawClassifierDataModule(root_dir, batch_size=4, num_workers=2, transform=None)

config = configparser.ConfigParser()
config.read('config.ini')
def setting():
    pd.set_option('display.max_colwidth', None)
    # Logging
    logs_directory = str(config['GA']['logs_dir_GA'])
    # Torch stuff
    seed = config.getint(section='Computation', option='seed')
    pl.seed_everything(seed=seed, workers=True)  # For reproducibility
    torch.set_float32_matmul_precision("medium")  # to make lightning happy
    num_workers = config.getint(section='Computation', option='num_workers')
    accelerator = config.get(section='Computation', option='accelerator')
    log_learning_rate=None
    batch_size=None
    # Get model parameters
    log_lr = log_learning_rate if log_learning_rate is not None else config.getfloat(section='Search Space', option='default_log_lr')
    lr = 10**log_lr
    bs = batch_size if batch_size is not None else config.getint(section='Search Space', option='default_bs')
    print(f"-----------The batch size of the data to be loaded in the model is: {bs}-----------")
    print(f"-----------The learning rate of the data to be loaded in the model is: {lr}-----------")
    
setting()

Seed set to 42


-----------The batch size of the data to be loaded in the model is: 4-----------
-----------The learning rate of the data to be loaded in the model is: 0.001-----------


In [3]:
# Model parameters
max_layers = 3
max_iter = int(config['GA']['max_iterations'])
# GA parameters
n_individuals = int(config['GA']['population_size'])
mating_pool_cutoff = float(config['GA']['mating_pool_cutoff'])
mutation_probability = float(config['GA']['mutation_probability'])

pop = Population(n_individuals=20, max_layers=max_layers, dm=dm, max_parameters=400_000)

In [4]:
pop.initial_poll()

Generating Population:   0%|          | 0/20 [00:00<?, ?it/s]

Generating Population: 100%|██████████| 20/20 [00:06<00:00,  3.26it/s]


In [ ]:
pop.train_generation(task='classification', lr=0.001, epochs=15, batch_size=32)

Training individual 8/20


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type                       | Params | Mode 
------------------------------------------------------------------------
0 | model            | Sequential                 | 4.2 M  | train
1 | loss_fn          | CrossEntropyLoss           | 0      | train
2 | accuracy         | MulticlassAccuracy         | 0      | train
3 | f1_score         | MulticlassF1Score          | 0      | train
4 | mcc              | MulticlassMatthewsCorrCoef | 0      | train
5 | conf_matrix      | MulticlassConfusionMatrix  | 0      | train
6 | conf_matrix_pred | MulticlassConfusionMatrix  | 0      | train
------------------------------------------------------------------------
4.2 M     Trainable params
0         Non-trainable params
4.2 M     Total params
16.793    Total estimated model params size (MB)
56        Modules in train mo

/home/vessel/anaconda3/envs/pynas/lib/python3.8/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (8) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Epoch 13:  12%|█▎        | 1/8 [00:01<00:07,  0.92it/s, v_num=43, train_loss=0.452, train_accuracy=0.730, train_f1_score=0.706, train_mcc=0.646]

In [19]:
pop.evolve(mating_pool_cutoff=mating_pool_cutoff, mutation_probability=0.85, k_best=1, n_random=3)

# Start Chain

Training of the model fresh created.

# To Be Rewritten with Bash Logic:

This part of the code train the whole population one individual at the time and can be performed on single individuals in parallel. Using bash we can run parallel training. 

In [ ]:
from api import api_call


# TODO: Implement the API call to evaluate the model and return the results.

# population loading from saved files
pop.load_dataframe(0)
pop.load_population(0)

# Train the models in the population           
idx = 0 # This must be input argument

nt = NASTrainer(population=pop, idx=idx, dm=dm, lr=1e-3, max_epochs=30)
nt.train()
nt.save_model()


# API to update the population with the results from the model training
result = api_call() # caller_api(nt.model)

# 1) Update the population with the results from the model training
fps = nt.results[0]['fps']
metric = nt.results[0]['test_mcc']
pop[idx].iou = nt.results[0]['test_mcc']
pop[idx].fps = nt.results[0]['fps']

# TODO> Implement the fitness function with more rigurous evaluation parameters (rn the fitness is not the best)
pop[idx].fitness = evaluator.weighted_sum_exponential(fps, metric)

# 2) Update the df. TODO: insert this logic in _update_df method.
pop.df.loc[idx, 'Fitness'] = pop[idx].fitness
pop.df.loc[idx, 'Metric'] = pop[idx].iou
pop.df.loc[idx, 'FPS'] = pop[idx].fps


pop.save_dataframe()
pop.save_population()




In [ ]:
if idx == len(pop) - 1:
    # Evolve the population
    pop.evolve(mating_pool_cutoff=mating_pool_cutoff, mutation_probability=0.85, k_best=1, n_random=3)

# Inference

Using the evaluated and saved model. We use the traced pytroch model (.pt) to load and execute inference.

In [ ]:
# Load the saved TorchScript model and test with a dummy input.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

save_path = "model_and_architecture.pt"
loaded_model = torch.jit.load(save_path, map_location=device)
loaded_model.eval()

# Ensure input is moved to the correct device
example_input = torch.randn(1, *dm.input_shape).to(device)
example_input = example_input.to(device)

with torch.no_grad():
    output = loaded_model(example_input)
print("Output from the loaded model:", output)